In [1]:
input_width = 160
input_height = 100
channels = 3
class_number = 12

In [ ]:
from keras.models import load_model
import keras.backend as K
def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

model = load_model('models/worm_single_nopooling_tiny_15.h5',custom_objects={'actual_acc':actual_acc})
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 39, 64)        12352     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 18, 64)        65600     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 16, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4719104   
_________________________________________________________________
dense_2 (Dense)              (None, 12)                6156      
Total para

In [ ]:
from screen_consts import WIDTH, HEIGHT, get_coordinates_from_direction
from grabscreen import grab_screen
from image_preproc import preproc_img
from getkeys import key_check
from image_preproc import prepare_image
import win32api, win32con
import time
import numpy as np

def mouse_down(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, x, y, 0, 0)

def mouse_up(x, y):
    win32api.SetCursorPos((x, y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, x, y, 0, 0)

    
from PIL import ImageGrab
def get_image():
    img = np.array(ImageGrab.grab())
    img = preproc_img(img)
    return img

#def get_image():
#    img = grab_screen()
#    img = preproc_img(img)
#    return img

class_number = 12
img = grab_screen()
scr_W, scr_H = len(img[0]), len(img)


print("Start/Pause action with Alt+T")
paused = True
    
while True:
    clock_prev = time.clock()
    keys = key_check()
    if "ALT" in keys and "T" in keys:
        paused ^= 1
        if paused:
            print("Stop action, press Alt+T to continue")
        else:
            print("Start action in 3 sec.")
            for i in list(range(3))[::-1]:
                time.sleep(1)
                print(i + 1)
            print("action!")
        time.sleep(1)
    if "C" in keys:
        offset = -70
        mouse_up(scr_W//2, scr_H//2+offset)
        mouse_down(scr_W//2, scr_H//2+offset)
        mouse_up(scr_W//2, scr_H//2+offset)
        
    img = get_image().reshape(-1,input_height, input_width,3)/255
        
    if not paused:              
        prediction_raw = model.predict(img)
        prediction = np.argmax(prediction_raw)
        x, y = get_coordinates_from_direction(prediction, scr_W, scr_H, class_number)
        mouse_up(x, y)
        #print(np.round(model.predict(prep_img), 2), prediction)
        #print('Took {} ms'.format(round((time.clock() - clock_prev)*1000)))

Start/Pause action with Alt+T
Start action in 3 sec.
3
2
1
action!
Stop action, press Alt+T to continue
Start action in 3 sec.
3
2
1
action!
Stop action, press Alt+T to continue


# VIEW INPUT

In [ ]:
%matplotlib notebook
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def grab_frame():
    return get_image()

#create two subplots
gridsize = (2, 1)
fig = plt.figure(figsize=(6, 8))
ax1 = plt.subplot2grid(gridsize, (0, 0))

#create two image plots
im1 = ax1.imshow(grab_frame(), cmap='Greys_r')

def update(i):    
    im1.set_data(grab_frame())

ani = FuncAnimation(plt.gcf(), update, interval=10)
plt.show()